* Incremental Update
* Off-Policy using importance Sampling
* Every visit Monte Carlo Policy Evaluation
* Weighted importance sampling

In [1]:
import random
import os
import time

All constants and tweakable parameters

In [2]:
#Discount-Factor
GAMMA = 0.9

#All possible actions defined
ACTION_UP = 'UP'
ACTION_DOWN = 'DOWN'
ACTION_LEFT = 'LEFT'
ACTION_RIGHT = 'RIGHT'

#Number of episodes
NUMBER_OF_EPISODES = 20

#Start and end of any episode
START_STATE = '00'
END_STATE = '15'

#Maximum allowed episode length
MAXIMUM_EPISODE_LENGTH = 100

#WAIT TIME
wait_time = 2

#Defining colors for highlighting important aspects
GREEN = lambda x: '\x1b[32m{}\x1b[0m'.format(x)
BLUE = lambda x: '\x1b[34m{}\x1b[0m'.format(x)
RED = lambda x: '\x1b[31m{}\x1b[0m'.format(x)

All datastructures need to traverse and store pertinent information has been defined below

In [3]:
all_states = ['00', '01', '02', '03',
          '04', '05', '06', '07',
          '08', '09', '10', '11',
          '12', '13', '14', '15']

immediate_state_rewards = {'00': -1,'01': -1,'02': -1,'03': -1,
                           '04': -1,'05': -1,'06': -1,'07': -1,
                           '08': -1,'09': -1,'10': -1,'11': -1,
                           '12': -1,'13': -1,'14': -1,'15': 0 
                          }

all_transitions =  {
    '00': {ACTION_UP : '00', ACTION_RIGHT : '01', ACTION_DOWN: '04', ACTION_LEFT: '00'},
    '01': {ACTION_UP : '01', ACTION_RIGHT : '02', ACTION_DOWN: '05', ACTION_LEFT: '00'},
    '02': {ACTION_UP : '02', ACTION_RIGHT : '03', ACTION_DOWN: '06', ACTION_LEFT: '01'},
    '03': {ACTION_UP : '03', ACTION_RIGHT : '03', ACTION_DOWN: '07', ACTION_LEFT: '02'},
    '04': {ACTION_UP : '00', ACTION_RIGHT : '05', ACTION_DOWN: '08', ACTION_LEFT: '04'},
    '05': {ACTION_UP : '01', ACTION_RIGHT : '06', ACTION_DOWN: '09', ACTION_LEFT: '04'},
    '06': {ACTION_UP : '02', ACTION_RIGHT : '07', ACTION_DOWN: '10', ACTION_LEFT: '05'},
    '07': {ACTION_UP : '03', ACTION_RIGHT : '07', ACTION_DOWN: '11', ACTION_LEFT: '06'},
    '08': {ACTION_UP : '04', ACTION_RIGHT : '09', ACTION_DOWN: '12', ACTION_LEFT: '08'},
    '09': {ACTION_UP : '05', ACTION_RIGHT : '10', ACTION_DOWN: '13', ACTION_LEFT: '08'},
    '10': {ACTION_UP : '06', ACTION_RIGHT : '11', ACTION_DOWN: '14', ACTION_LEFT: '09'},
    '11': {ACTION_UP : '07', ACTION_RIGHT : '11', ACTION_DOWN: '15', ACTION_LEFT: '10'},
    '12': {ACTION_UP : '08', ACTION_RIGHT : '13', ACTION_DOWN: '12', ACTION_LEFT: '12'},
    '13': {ACTION_UP : '09', ACTION_RIGHT : '14', ACTION_DOWN: '13', ACTION_LEFT: '12'},
    '14': {ACTION_UP : '10', ACTION_RIGHT : '15', ACTION_DOWN: '14', ACTION_LEFT: '13'},
    '15': {ACTION_UP : '15', ACTION_RIGHT : '15', ACTION_DOWN: '15', ACTION_LEFT: '15'},
}

all_state_action_value_pairs = {
    '00': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '01': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '02': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '03': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '04': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '05': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '06': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '07': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '08': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '09': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '10': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '11': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '12': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '13': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '14': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '15': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
}

total_state_action_visits = {
    '00': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '01': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '02': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '03': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '04': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '05': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '06': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '07': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '08': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '09': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '10': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '11': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '12': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '13': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '14': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
    '15': {ACTION_UP : 0, ACTION_RIGHT : 0, ACTION_DOWN: 0, ACTION_LEFT: 0},
}

I define my target policy as described in the slides

In [4]:
target_policy =  {
    '00': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '01': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '02': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '03': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '04': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '05': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '06': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '07': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '08': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '09': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '10': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '11': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '12': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '13': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '14': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
    '15': {ACTION_UP : 0.1, ACTION_RIGHT : 0.4, ACTION_DOWN: 0.4, ACTION_LEFT: 0.1},
}

Following is the definition of my behavior policy

In [5]:
behavior_policy =  {
    '00': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '01': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '02': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '03': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '04': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '05': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '06': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '07': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '08': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '09': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '10': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '11': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '12': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '13': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '14': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
    '15': {ACTION_UP : 0.25, ACTION_RIGHT : 0.25, ACTION_DOWN: 0.25, ACTION_LEFT: 0.25},
}

General purpose functions

In [6]:
def printStateActionValuePairs():
    print(" \t", ACTION_UP, "\t", ACTION_RIGHT, "\t", ACTION_DOWN, "\t", ACTION_LEFT)
    for state in all_states:
        print(state, "\t", "%.2f" % all_state_action_value_pairs[state][ACTION_UP], "\t", "%.2f" % all_state_action_value_pairs[state][ACTION_RIGHT], "\t", "%.2f" % all_state_action_value_pairs[state][ACTION_DOWN], "\t", "%.2f" % all_state_action_value_pairs[state][ACTION_LEFT],)
    print("\n\n")

Functions for generating episodes

In [7]:
def chooseActionForBehaviorPolicy():
    random_throw = random.uniform(0, 1)
    if random_throw < 0.25:
        return ACTION_UP
    elif random_throw < 0.5:
        return ACTION_RIGHT
    elif random_throw < 0.75:
        return ACTION_DOWN
    else:
        return ACTION_LEFT

def generateEpisodeForBehaviorPolicy():
    current_state = START_STATE
    states_in_episode = []
    actions_in_episode = []
    while (current_state != END_STATE) & (len(states_in_episode) < MAXIMUM_EPISODE_LENGTH):
        states_in_episode.append(current_state)
        action = chooseActionForBehaviorPolicy()
        actions_in_episode.append(action)
        
        #os.system('clear')
        #printGridWorld(states_in_episode, actions_in_episode)
        #time.sleep(wait_time)
        
        current_state = all_transitions.get(current_state).get(action)
        
    if current_state == END_STATE:
        states_in_episode.append(END_STATE)
        actions_in_episode.append(chooseActionForBehaviorPolicy())
    
    #os.system('clear')
    #printGridWorld(states_in_episode, actions_in_episode)
    #time.sleep(wait_time)
    
    return states_in_episode, actions_in_episode

In [8]:
def chooseActionForTargetPolicy():
    random_throw = random.uniform(0, 1)
    if random_throw < 0.10:
        return ACTION_UP
    elif random_throw < 0.5:
        return ACTION_RIGHT
    elif random_throw < 0.90:
        return ACTION_DOWN
    else:
        return ACTION_LEFT

def generateEpisodeForTargetPolicy():
    current_state = START_STATE
    states_in_episode = []
    actions_in_episode = []
    while (current_state != END_STATE) & (len(states_in_episode) < MAXIMUM_EPISODE_LENGTH):
        states_in_episode.append(current_state)
        action = chooseActionForTargetPolicy()
        actions_in_episode.append(action)
        
        #os.system('clear')
        #printGridWorld(states_in_episode, actions_in_episode)
        #time.sleep(wait_time)
        
        current_state = all_transitions.get(current_state).get(action)
        
    if current_state == END_STATE:
        states_in_episode.append(END_STATE)
        actions_in_episode.append(chooseActionForTargetPolicy())
    
    #os.system('clear')
    #printGridWorld(states_in_episode, actions_in_episode)
    #time.sleep(wait_time)
    
    return states_in_episode, actions_in_episode

The following part now implements and runs the algorithm that we have been talking about so far.

![Algorithm followed](images/Algo.jpg)

In [9]:
for episodes_iterator in range(NUMBER_OF_EPISODES):
    states, actions = generateEpisodeForBehaviorPolicy()
    returns = 0.0
    W = 1
    #print("Length is " , str(len(states)))
    for step in range(len(states) - 1, -1, -1):
        returns = (GAMMA * returns) + immediate_state_rewards[states[step]]
        total_state_action_visits[states[step]][actions[step]] = total_state_action_visits[states[step]][actions[step]] + W
        all_state_action_value_pairs[states[step]][actions[step]] = all_state_action_value_pairs[states[step]][actions[step]] + (W/total_state_action_visits[states[step]][actions[step]]) * (returns - all_state_action_value_pairs[states[step]][actions[step]])
        W = W * ((target_policy[states[step]][actions[step]])/behavior_policy[states[step]][actions[step]])
        if W == 0:
            break
            
printStateActionValuePairs()

 	 UP 	 RIGHT 	 DOWN 	 LEFT
00 	 -7.79 	 -7.66 	 -6.82 	 -7.31
01 	 -7.72 	 -7.63 	 -6.92 	 -8.17
02 	 -4.96 	 -6.09 	 -4.74 	 -7.11
03 	 -6.14 	 -5.51 	 -4.38 	 -5.15
04 	 -7.30 	 -6.46 	 -6.60 	 -5.90
05 	 -7.18 	 -4.76 	 -6.30 	 -7.23
06 	 -6.01 	 -3.37 	 -3.79 	 -9.39
07 	 -5.36 	 -5.64 	 -2.85 	 -3.61
08 	 -7.93 	 -4.30 	 -6.63 	 -8.28
09 	 -5.40 	 -3.32 	 -7.03 	 -4.76
10 	 -4.71 	 -3.19 	 -3.21 	 -4.60
11 	 -6.90 	 -2.13 	 -1.00 	 -3.60
12 	 -8.50 	 -6.23 	 -4.59 	 -7.18
13 	 -7.44 	 -5.45 	 -5.31 	 -7.34
14 	 -5.28 	 -1.00 	 -3.63 	 -5.40
15 	 0.00 	 0.00 	 0.00 	 0.00



